# Unicorn Companies Data Analysis

In this notebook, we will analyze the Unicorn Companies dataset using Snowflake SQL. The analysis includes loading the data, cleaning it, and addressing various business questions using SQL queries.

The dataset contains information about unicorn companies, including their valuations, industries, and country of origin.

# 01. Loading data

In [ ]:
USE DATABASE MAIN

In [ ]:
CREATE OR REPLACE TABLE TBL AS
SELECT
    Industry,
    Company,
    Country,
    City,
    Total_Raised,
    Valuation,
    Founded_Year,
    Date_Joined
FROM
    UNICORN_COMPANIES;

# 02. Data Cleaning

Transformation in

In [ ]:
UPDATE TBL
SET 
    INDUSTRY = REPLACE(INDUSTRY, '"', ''),
    VALUATION = REPLACE(VALUATION, '$', ''),
    TOTAL_RAISED = REPLACE(REPLACE(TOTAL_RAISED, '$', ''), 'None', '0');

In [ ]:
DELETE FROM TBL
WHERE 
    Industry NOT IN (
        'Artificial intelligence', 
        'Other', 
        'Fintech', 
        'Internet software & services',
        'Supply chain, logistics, & delivery',
        'Data management & analytics',
        'E-commerce & direct-to-consumer',
        'Edtech',
        'Hardware',
        'Health',
        'Auto & transportation',
        'Cybersecurity', 
        'Mobile & telecommunications',
        'Consumer & retail',
        'Travel'
    )
    OR FOUNDED_YEAR = 'None'
    OR YEAR(CAST(Date_Joined AS DATE)) < YEAR(CAST(CONCAT(FOUNDED_YEAR, '-01-01') AS DATE));

In [ ]:
UPDATE TBL
SET TOTAL_RAISED = 
    COALESCE(
        CASE
            WHEN TOTAL_RAISED IS NULL OR TRIM(TOTAL_RAISED) = '' THEN '0'
            WHEN RIGHT(TOTAL_RAISED, 1) = 'B' THEN REPLACE(TOTAL_RAISED, 'B', '')
            WHEN RIGHT(TOTAL_RAISED, 1) = 'M' THEN 
                CAST(REPLACE(REPLACE(TOTAL_RAISED, ',', ''), 'M', '') AS FLOAT) / 1000
            WHEN RIGHT(TOTAL_RAISED, 1) = 'K' THEN 
                CAST(REPLACE(REPLACE(TOTAL_RAISED, ',', ''), 'K', '') AS FLOAT) / 1000000
            ELSE REPLACE(REPLACE(TOTAL_RAISED, ',', ''), '$', '')
        END, 
        '0'
    );

In [ ]:
ALTER TABLE TBL
DROP COLUMN FOUNDED_YEAR, DATE_JOINED;

# 03. Analyzing Data

With the data cleaned, we can analyze it to answer business questions. We will focus on the industry and country versus valuation and total raised variables.

In [ ]:
SELECT * FROM TBL
LIMIT 6;

What is the average company valuation for each industry? How do these industries rank based on their average valuation?

In [ ]:
SELECT
    Industry,
    AVG(Valuation) AS AvgValuation
FROM
    TBL
GROUP BY
    Industry
ORDER BY
    AvgValuation DESC;

Which industry has the highest total raised?

In [ ]:
SELECT
    Industry,
    SUM(Total_Raised) AS Total_Raised
FROM
    TBL
GROUP BY
    Industry
ORDER BY
    Total_Raised DESC;

Which are the top 10 cities in the United States with the highest average company valuation? How many companies are there in each of these cities?

In [ ]:
WITH CityAverageValuation AS (
    SELECT
        City,
        AVG(Valuation) AS AvgValuation,
        COUNT(*) AS NumberOfCompanies
    FROM
        TBL
    WHERE
        Country = 'United States'
    GROUP BY
        City
)
SELECT
    City,
    AvgValuation,
    NumberOfCompanies
FROM
    CityAverageValuation
ORDER BY
    NumberOfCompanies DESC
LIMIT 10;

What are the top companies in each industry based on the total amount raised, and how do they rank within their industry?

In [ ]:
WITH RankedCompanies AS (
    SELECT
        Company,
        Industry,
        Total_Raised,
        AVG(TBL.Total_Raised) OVER (PARTITION BY TBL.Industry) AS AvgTotalRaised,
        RANK() OVER (PARTITION BY Industry ORDER BY Total_Raised DESC) AS TotalRaisedRank
    FROM
        TBL
)
SELECT
    Company,
    Industry,
    Total_Raised,
    AvgTotalRaised,
    TotalRaisedRank
FROM
    RankedCompanies
WHERE
    TotalRaisedRank <= 3;

Which are the top 3 companies by valuation within each industry? What is their valuation relative to the average valuation in their industry?

In [ ]:
WITH RankedCompanies AS (
    SELECT
        TBL.Company,
        TBL.Industry,
        TBL.Valuation,
        AVG(TBL.Valuation) OVER (PARTITION BY TBL.Industry) AS AvgValuation,
        RANK() OVER (PARTITION BY TBL.Industry ORDER BY TBL.Valuation DESC) AS ValuationRank
    FROM
        TBL
)
SELECT
    Company,
    Industry,
    Valuation,
    AvgValuation,
    ValuationRank
FROM
    RankedCompanies
QUALIFY
    ValuationRank <= 3;


What is the correlation between total raised and valuation by country? Calculated for top countries by number of comapnies (HAVING COUNT(*) > x).

In [ ]:
SELECT
    Country,
    CORR(Total_Raised, Valuation) AS Correlation
FROM
    TBL
GROUP BY
    Country
HAVING
    COUNT(*) > 50
ORDER BY
    Correlation DESC;